## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
import os
import sys
import inspect

sys.path.insert(1, os.path.join(sys.path[0], '..'))
import config

from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
f = open(r"C:\Users\efren\OneDrive\Desktop\DataBootCamp\Github\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df = pd.read_csv(f)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Patricio,US,28.0170,-97.5169,82.81,53,97,2.55,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,76.33,70,0,17.20,clear sky
2,2,Arman,RU,59.7000,150.1667,5.02,88,100,19.46,light snow
3,3,East London,ZA,-33.0153,27.9116,69.42,88,90,11.86,light rain
4,4,Mount Gambier,AU,-37.8333,140.7667,58.91,88,90,10.36,overcast clouds


In [33]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 75


In [34]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,East London,ZA,-33.0153,27.9116,69.42,88,90,11.86,light rain
4,4,Mount Gambier,AU,-37.8333,140.7667,58.91,88,90,10.36,overcast clouds
7,7,Sao Felix Do Xingu,BR,-6.6447,-51.9950,74.57,94,97,1.63,moderate rain
11,11,Ushuaia,AR,-54.8000,-68.3000,55.06,41,40,18.41,light rain
13,13,Punta Arenas,CL,-53.1500,-70.9167,57.31,33,40,24.16,scattered clouds
14,14,Pangkalanbuun,ID,-2.6833,111.6167,73.96,94,96,2.55,overcast clouds
15,15,Bluff,NZ,-46.6000,168.3333,66.79,73,15,2.04,few clouds
17,17,Ponta Delgada,PT,37.7333,-25.6667,64.51,82,40,6.91,scattered clouds
22,22,Bredasdorp,ZA,-34.5322,20.0403,56.28,65,98,7.20,overcast clouds
39,39,Airai,TL,-8.9266,125.4092,63.01,79,100,0.81,overcast clouds


In [52]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                207
City                   207
Country                203
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current Description    207
dtype: int64

In [53]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                203
City                   203
Country                203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [54]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,East London,ZA,69.42,light rain,-33.0153,27.9116,
4,Mount Gambier,AU,58.91,overcast clouds,-37.8333,140.7667,
7,Sao Felix Do Xingu,BR,74.57,moderate rain,-6.6447,-51.9950,
11,Ushuaia,AR,55.06,light rain,-54.8000,-68.3000,
13,Punta Arenas,CL,57.31,scattered clouds,-53.1500,-70.9167,
14,Pangkalanbuun,ID,73.96,overcast clouds,-2.6833,111.6167,
15,Bluff,NZ,66.79,few clouds,-46.6000,168.3333,
17,Ponta Delgada,PT,64.51,scattered clouds,37.7333,-25.6667,
22,Bredasdorp,ZA,56.28,overcast clouds,-34.5322,20.0403,
39,Airai,TL,63.01,overcast clouds,-8.9266,125.4092,


In [76]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city. 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [78]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [81]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [84]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [86]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))